# Intro

We are wrangling up some data from simulated APT activity that was captured on a mock production network with the efforts of creating a "realistic, semi-synthetic" dataset. I will document some of the process that I undertook, as a decent amount of it was exploratory, as well as covering the changes I had to make with the original dataset.

To reduce scope, yet still covering both the axes of host and network logs, I will just be wrangling the Netflow data, Linux auth+audit logs, and Windows Security Events.

The source for this dataset can be found here: https://doi.org/10.1016/j.comnet.2023.109688

The original data/ can be found here: https://www.kaggle.com/datasets/ernie55ernie/unraveled-advanced-persistent-threats-dataset/data

Due to the size of the data (835 MB ZIP, decompressed to 4.43 GB of plaintext and binary data) the download will take some time, but I used the following command:

```zsh
  curl -L -o ~/Downloads/unraveled-advanced-persistent-threats-dataset.zip\
    https://www.kaggle.com/api/v1/datasets/download/ernie55ernie/unraveled-advanced-persistent-threats-dataset
```

While cleaning up the data, we should keep in mind our hypothesis and trim away anything that probably won't contribute to proving the null or alternative.

Hypothesis: Choose one
- APTs adjust TTPs in response to defensive measures and signs of detection.
- We can better detect APTs based on their TTPs versus specific artifacts
  - Testing the highest scoring/most imporant features of a model

# Setup

Below I threw together a few helper functions to solve a couple problems I ran into when trying to load CSVs.

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import chardet

%matplotlib inline

def get_encoding(path):
    with open(path, 'rb') as f:
        raw = f.read(4096)  # read first 4 KB

        # Use chardet lib to detect the encoding
        result = chardet.detect(raw)

        return result['encoding']

def get_files_recurse(path):
    result = []
    
    # For each file, append its full path to a list
    for root, dirs, files in os.walk(path):
        for file in files:
            fullpath = os.path.join(root, file)
            result.append(fullpath)
            
    return result

def load_all_csv(path, sep=',', recurse=False, verbose=False, encoding='auto'):
    files = [path + x for x in os.listdir(path)] if not recurse else get_files_recurse(path)
    d = dict()
    
    # For each file, check its encoding scheme, then store as DF in dict with fullpath as key
    for f in files:
        if not os.path.isfile(f):
            continue
        
        if verbose:
            print(f)
        
        enc = get_encoding(f) if encoding == 'auto' else encoding
        d[f] = pd.read_csv(f, delimiter=sep, encoding=enc)
        
    # Concatenate all DFs in the dictionary, ignoring the indexes so they don't collide
    df = pd.concat(d.values(), ignore_index=True)
        
    return df

# Network Logs

## Netflow

In [ ]:
path = '../data/unraveled-apt/network-flows/'

df = load_all_csv(path, recurse=True)

In [ ]:
dropme = [
    # Identifiers
    'fgid', 'id', ' id',
    
    # Redundant and unneeded Layer 2/3 info
    'src_oui', 'dst_oui', 'tunnel_id', 'ip_version',
    'vlan_id',
    
    # Sparse application metadata
    'requested_server_name', 'client_fingerprint', 
    'content_type', 'application_is_guessed',
    
    # Redundant bidirectional stats
    'bidirectional_min_ps', 'bidirectional_mean_ps', 
    'bidirectional_stddev_ps', 'bidirectional_max_ps',
    'bidirectional_min_piat_ms', 'bidirectional_mean_piat_ms',
    'bidirectional_stddev_piat_ms', 'bidirectional_max_piat_ms',
    
    # Redundant bidirectional TCP flags (keep directional)
    'bidirectional_syn_packets', 'bidirectional_cwr_packets',
    'bidirectional_ece_packets', 'bidirectional_urg_packets',
    'bidirectional_ack_packets', 'bidirectional_psh_packets',
    'bidirectional_rst_packets', 'bidirectional_fin_packets',
    
    # Potentially redundant timing
    'src2dst_last_seen_ms', 'dst2src_last_seen_ms'
]

reduced_df = df.drop(columns=dropme)

In [ ]:
# Replace null values
reduced_df['Signature'] = reduced_df['Signature'].fillna('Normal')

In [ ]:
reduced_df.info(memory_usage='deep')

Even with dropping a decent amount of columns, we still have a DataFrame that takes up over 6GB of memory. All of the datatypes appear to just be the default `int64`/`object`, so we can make some changes to that and save a fair amount.

I applied the methodology commented into the code below:

In [ ]:
# We're only working with whole numbers here.
floats = reduced_df.dtypes[reduced_df.dtypes == 'float64'].index
reduced_df[floats] = reduced_df[floats].astype('int64')

# Storing some frequently referenced sets of values
int_cols = reduced_df.dtypes[reduced_df.dtypes == 'int64'].index
maxes = reduced_df[int_cols].max()


# I used unsigned ints because none of the values are negative.
# uint16 = 0-65535
uint16 = maxes < 65536
uint16 = uint16[uint16].index

# uint32 = 0-4294967295
uint32 = maxes < 4294967296  # includes uint16 cols, but we will do that type change after this one
uint32 = uint32[uint32].index

# For these, I manually checked how many nunique() they had, and it was on the lower end.
categories = [
    'src_ip', 'dst_ip', 'src_mac', 
    'dst_mac', 'expiration_id', 'application_name', 
    'user_agent', 'server_fingerprint', 'Activity', 
    'DefenderResponse', 'Signature', 'Stage',
    'application_category_name'
]

In [ ]:
reduced_df[categories].nunique().sort_values()

In [ ]:
reduced_df[categories] = reduced_df[categories].astype('category')
reduced_df[float32] = reduced_df[float32].astype('float32')
reduced_df[uint32] = reduced_df[uint32].astype('uint32')
reduced_df[uint16] = reduced_df[uint16].astype('uint16')

reduced_df['flow_start'] = pd.to_datetime(reduced_df['bidirectional_first_seen_ms'], unit='ms')
reduced_df['flow_end'] = pd.to_datetime(reduced_df['bidirectional_last_seen_ms'], unit='ms')

In [ ]:
import gc

# lets free up some memory
del df
gc.collect()

In [ ]:
# Move start and end timestamps if they are not already there
reduced_df = reduced_df[['flow_start', 'flow_end'] + 
                        [c for c in reduced_df.columns if c not in ['flow_start', 'flow_end']]]

reduced_df.head()

In [ ]:
reduced_df.info(memory_usage='deep')

In [ ]:
# Export as pickle to save all that hard work we did converting datatypes
reduced_df.to_pickle('../data/cleaned/netflow.pkl')

These Netflow logs should be ready for us to play around with further.

In [ ]:
reduced_df = pd.read_pickle('../data/cleaned/netflow.pkl')

# Linux Host Logs

## `audit`

In [ ]:
path = os.path.split(os.getcwd())[0] + '/data/unraveled-apt/host-logs/audit/'
audit_df = load_all_csv(path, sep=';')

In [ ]:
audit_df.shape

On the last row, there appears to be some preceeding whitespace in the LogEvent column. Lets handle that:

In [ ]:
for col in audit_df.columns:
    try:
        audit_df[col] = audit_df[col].str.strip()
    except:
        continue

With that out of the way, we will need to address the log message that is sometimes nested under the `msg` field.

In [ ]:
audit_df.LogEvent.iloc[111]

```sh
type=USER_START 
ts=1621862701.432 
...
# I still want to keep this mostly intact
msg=\'op=PAM:session_open acct="root" exe="/usr/sbin/cron" hostname=? addr=? terminal=cron res=success\'
```

I plan to keep this by extracting `msg` out of the string, processing it separately from the rest of the log, then throwing `msg` into the rest of the log as a column.

In [ ]:
msg_df = audit_df.LogEvent.str.extract(r"msg=('.*')")
no_msg = audit_df.LogEvent.str.replace(r"msg=('.*')", repl='', regex=True)

This is what we extracted:

In [ ]:
msg_df.head(10)

Here is what the `no_msg` series looks like now. We can proceeed with converting this into a DataFrame, then concatenating `msg_df` to it.

In [ ]:
print("Message removed:", no_msg.iloc[111])
print("Original log:   ", audit_df.LogEvent.iloc[111])

We'll store our final result in a var called `logs`. Very descriptive.

In [ ]:
logs = no_msg.str.split()

In [ ]:
logs.head()

In [ ]:
logs.iloc[logs.shape[0]-1]  # We are doing it this way because I like how it formats the text better. No judging!

In [ ]:
expand_logs = logs.apply(lambda x: {split_field[0]: split_field[1] for split_field in [log_field.split('=') for log_field in x]}).to_dict()
list(expand_logs.items())[:2] # logs are expanded to a dictionary of dictionaries

In [ ]:
log_df = pd.DataFrame(expand_logs).T
print(log_df.head(6))

del expand_logs

In [ ]:
log_df.columns  # no msg column

In [ ]:
log_df['ts'] = pd.to_datetime(log_df['ts'].str.replace('.', ''), unit='ms')
log_df['ts'].head()

In [ ]:
# Create DataFrame of labeled audit log data
labeled_audit_df = pd.concat([
        msg_df,  # contains the retained msg field
        log_df,  # contains the rest of the log, parsed
        audit_df[audit_df.columns[1:]]  # slice off first column, since we just expanded that.
        # This will give us LogEvent expanded out into more columns as well as the labels.
    ], 
    axis=1)

labeled_audit_df.rename({0: 'msg'}, inplace=True, axis=1)

In [ ]:
# reordering the columns to put the msg field in position 11
labeled_audit_df = labeled_audit_df[labeled_audit_df.columns[1:].insert(11, 'msg')]

In [ ]:
labeled_audit_df['Signature'] = labeled_audit_df['Signature'].fillna('Normal')

In [ ]:
# We can save a lot of memory just by changing some columns to type 'category'
categories = labeled_audit_df.columns[labeled_audit_df.nunique() < 100]
labeled_audit_df[categories] = labeled_audit_df[categories].astype('category')

In [ ]:
# Over half the values in each of these columns == null
dropme = labeled_audit_df.columns[labeled_audit_df.notna().sum() / labeled_audit_df.shape[0] < 0.50]
dropme

In [ ]:
labeled_audit_df.drop(columns=dropme, inplace=True)

In [ ]:
labeled_audit_df.info(memory_usage='deep')

In [ ]:
labeled_audit_df[['pid', 'ses']] = labeled_audit_df[['pid', 'ses']].fillna(0)

labeled_audit_df.pid = labeled_audit_df.pid.astype('uint32')
labeled_audit_df.tsid = labeled_audit_df.tsid.astype('uint32')
labeled_audit_df.ses = labeled_audit_df.ses.astype('uint32')

In [ ]:
# Saving our work to save me some time.
labeled_audit_df.to_pickle('../data/cleaned/audit.pkl')

## `auth`

In [ ]:
path = os.path.split(os.getcwd())[0] + '/data/unraveled-apt/host-logs/auth/'
auth_df = load_all_csv(path, sep='|')

In [ ]:
for col in auth_df.columns[1:]:
    print(auth_df[col].value_counts(), end=f'\n{'-'*20}\n')

In [ ]:
auth_df.LogEvent.iloc[[5, 10, 15, 20, 25, 100, 200, 300, 1000, 2000]].values

In [ ]:
logs = auth_df.LogEvent.apply(lambda x: x.split(' ', maxsplit=5))
logs.head().values

In [ ]:
df = pd.DataFrame(data=logs.tolist(), columns=['month', 'day', 'time', 'hostname', 'app', 'msg'])


In [ ]:
df['ts'] = "2021-"+df['month']+"-"+df['day']+" "+df['time']
df['ts'] = pd.to_datetime(df['ts'])

In [ ]:
# Drop redundant date cols and make ts col 0
df.drop(['month', 'day', 'time'], axis=1, inplace=True, errors='ignore')
df = df[df.columns[:-1].insert(0, 'ts')]

In [ ]:
df.head()

In [ ]:
tmp = df['app'].str.split('[')

In [ ]:
tmp = tmp.apply(lambda x: [e.strip(']:') for e in x])
tmp = tmp.apply(lambda x: x+[0] if len(x) == 1 else x)

In [ ]:
tmp

In [ ]:
tmp = pd.DataFrame(tmp.tolist(), columns=['app','pid'])

In [ ]:
tmp.head()

In [ ]:
df['app'] = tmp['app']
df['pid'] = tmp['pid']


In [ ]:
df = df[['ts','hostname','app','pid','msg']]
df.head()

In [ ]:
del tmp

In [ ]:
df.msg = df.msg.apply(lambda x: x.strip())

In [ ]:
categories = df.columns[df.nunique() < 100]
categories

In [ ]:
df[categories] = df[categories].astype('category')

In [ ]:
df.pid = df.pid.astype('uint32')

In [ ]:
df.info(memory_usage='deep')

Now to finally add the labels!

In [ ]:
labels = auth_df.columns[1:]
df[labels] = auth_df[labels].astype('category')

In [ ]:
df.head()

In [ ]:
df.info(memory_usage='deep')

In [ ]:
df.to_pickle('../data/cleaned/auth.pkl')

import gc

del df, auth_df
gc.collect()

## Combined

In [ ]:
combined_linux_host_df = pd.concat([audit_df, auth_df], ignore_index=True)

In [ ]:
combined_linux_host_df.info()

-----

# Windows Host Logs

## Security.evtx

In [ ]:
df = load_all_csv(path='../data/unraveled-apt/host-logs/windows/', encoding='utf-8')

In [ ]:
df.head()

In [ ]:
df['Signature'] = df['Signature'].fillna('Normal')

Perusing through the data seems to show that the cleaning messed up event ID 4625, as it was probably formatted slightly differently. We'll fix this just by using `fillna` and moving some columns around

In [ ]:
tmp = df[df.EventID == 4625].copy()
tmp.head()

In [ ]:
tmp.LogMessage = tmp.Activity
tmp.Activity = tmp.Activity.apply(lambda x:"Normal")
tmp[['Stage', 'DefenderResponse']] = tmp[['Stage', 'DefenderResponse']].fillna('Benign')

In [ ]:
tmp.head()

There we go, that looks better.

In [ ]:
df[df.EventID == 4625] = tmp

In [ ]:
df[df.EventID == 4625].head()

In [ ]:
df.info(memory_usage='deep')

In [ ]:
df.nunique().sort_values()

In [ ]:
df.DateTime = pd.to_datetime(df.DateTime)

In [ ]:
categories = df.columns[df.nunique() < 1000]
categories

In [ ]:
df[categories] = df[categories].astype('category')

In [ ]:
df.info(memory_usage='deep')

In [ ]:
df[df.LogMessage.isna()].head()

Let's check both of the NaN fields to make sure they only contain nulls.

In [ ]:
df[df.LogMessage.isna()][['Signature', 'LogMessage']].isna().all()

In [ ]:
df.to_pickle('../data/cleaned/win-security.pkl')

In [ ]:
df = pd.read_pickle('../data/cleaned/win-security.pkl')

In [ ]:
df.head()